# IBM Think 2020 Code Cafe - Tic Tac Toe with DeepMind OpenSpiel

![https://nextgrid.ai/wp-content/uploads/2020/05/ibmthink.png](https://nextgrid.ai/wp-content/uploads/2020/05/ibmthink.png)

# What is Reinforcement Learning  (RL)?

According  to Wikipedia:
> Reinforcement learning (RL) is an area of machine learning concerned with how software agents ought to take actions in an environment in order to maximize the notion of cumulative reward.

**In other words: the goal of RL is to learn some program how to behave to get as much reward as possible.**


## Basic vocabulary

Take a look at the following picture from the *Reinforcement Learning: An Introduction* book by Richard S. Sutton and Andrew Barto:  
![alt text](https://www.kdnuggets.com/images/reinforcement-learning-fig1-700.jpg)

An **agent** takes an **action** based on the current **state** and gets a **reward** from the **environment ** (please note that reward can be delayed and in timestep *t* agent can receive some reward from an action taken 20 timesteps ago).
Agent takes actions until it reaches the terminal state - which indicates the end of the **episode**.


## Tic-Tac-Toe example
![alt text](https://images.squarespace-cdn.com/content/v1/54f74f23e4b0952b4e0011c0/1580269334204-W1N8ATYATHA6XP02YVSY/ke17ZwdGBToddI8pDm48kGtxPgPaOBG5VTwzK0O3JPx7gQa3H78H3Y0txjaiv_0fDoOvxcdMmMKkDsyUqMSsMWxHk725yiiHCCLfrh8O1z5QHyNOqBUUEtDDsRWrJLTmYfwwyaF2qdqpAEW-vwkS-q9yrvcVcBFNcMZ7RZJD-G-L7L3_iLqMJNwF1D5UY19g/tictac.png?format=400w)

In a simple game Tic-Tac-Toe, two players play against each other.  
One agent puts `X` or `O` in a certain board slot (takes an action) based on the current board state. Then the other agent (environment from the perspective of the first agent) takes its action based on the new board state. And so on.  
The game finishes when one agent has 3 `X` or `O` in a row.


# Q-Value
One of the simplest and the most powerful concepts in RL is **Q-value**.  
It is an estimation of **"how good it to take certain action in a given situation"**.  
For example, you can think of it like of wondering is it good to be Bill Gates and spent money on the charity or deciding if you should go to the safe boat on the Titanic.  

In the state of the art RL algorithms, we train a Q-Value function which approximates how good it is to be in a given state and take certain action - it is the fundament of the Deep Q-Network (DQN) - an algorithm which beats human-level performance in the Atari games.

# Import all neccessary libraries.
Let's prepare the environment for our playthrough

In [54]:
import sys
sys.path.append('/home/jupyter/open_spiel')
sys.path.append('/home/jupyter/open_spiel/build/python')  # for pyspiel.so

In [55]:
# verify that Python can find the open_spiel & pyspiel modules
import importlib
assert importlib.util.find_spec("open_spiel") is not None
assert importlib.util.find_spec("pyspiel") is not None

In [56]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import sys
from absl import app
from absl import flags
import numpy as np
import tensorflow.compat.v1 as tf

from open_spiel.python import rl_environment
from open_spiel.python.algorithms import dqn
from open_spiel.python.algorithms import random_agent
from open_spiel.python.algorithms import tabular_qlearner
from open_spiel.python.bots import human

game = "tic_tac_toe"
num_players = 2
env = rl_environment.Environment(game)
num_actions = env.action_spec()["num_actions"]


def pretty_board(time_step):
    """Returns the board in `time_step` in a human readable format."""
    info_state = time_step.observations["info_state"][0]
    x_locations = np.nonzero(info_state[9:18])[0]
    o_locations = np.nonzero(info_state[18:])[0]
    board = np.full(3 * 3, ".")
    board[x_locations] = "X"
    board[o_locations] = "0"
    board = np.reshape(board, (3, 3))
    return board

def eval_against_random_bots(env, trained_agents, random_agents, num_episodes):
    """Evaluates `trained_agents` against `random_agents` for `num_episodes`."""
    num_players = len(trained_agents)
    sum_episode_rewards = np.zeros(num_players)
    for player_pos in range(num_players):
        cur_agents = random_agents[:]
        cur_agents[player_pos] = trained_agents[player_pos]
        for _ in range(num_episodes):
            time_step = env.reset()
            episode_rewards = 0
            while not time_step.last():
                player_id = time_step.observations["current_player"]
                agent_output = cur_agents[player_id].step(time_step, is_evaluation=True)
                action_list = [agent_output.action]
                time_step = env.step(action_list)
                episode_rewards += time_step.rewards[player_pos]
            sum_episode_rewards[player_pos] += episode_rewards
    return sum_episode_rewards / num_episodes


# Define two agents

It is a simple problem with less than 9^3 different states so we can calculate Q-Value directly and we don't need to use it approximation from DQN.

In [57]:
tabular_q_agent_0 = tabular_qlearner.QLearner(
    player_id=0, num_actions=num_actions)
tabular_q_agent_1 = tabular_qlearner.QLearner(
    player_id=1, num_actions=num_actions)

agents = [tabular_q_agent_0, tabular_q_agent_1]

# Train agents for 100 episodes
This agent should be imperfect but better than random one.

In [58]:
train_episodes = 100

# Train agent
for ep in range(train_episodes):
    time_step = env.reset()
    while not time_step.last():
        player_id = time_step.observations["current_player"]
        agent_output = agents[player_id].step(time_step)
        action_list = [agent_output.action]
        time_step = env.step(action_list)

    # Episode is over, step all agents with final info state.
    for agent in agents:
        agent.step(time_step)
print("Training is over.")

Training is over.


# Evaluate first agent playing agains random agent for 1000 episodes
Reward -1 means loss  
Reward 0 means draw  
Reward 1 mean win

In [59]:
# Evaluate against random agent
random_agents = [
    random_agent.RandomAgent(player_id=idx, num_actions=num_actions)
    for idx in range(num_players)
]
r_mean = eval_against_random_bots(env, agents, random_agents, 1000)
print("Mean episode rewards: %s" % r_mean[0])

Mean episode rewards: 0.327


# Lets play with an agent after 100 games of experience
You can play against such weak agent. It should be easy to win with it. 

#### Important
You press Enter in input field to see avalible moves


In [60]:
human_player = 1
human_agent = human.HumanBot()

print("You are playing as X (you are second)" ) 
time_step = env.reset()
while not time_step.last():
    player_id = time_step.observations["current_player"]
    print("\n%s" % pretty_board(time_step))

    if player_id == human_player:
        agent_out = agents[human_player].step(time_step, is_evaluation=True)
        action = human_agent.step(env._state)
    else:
        agent_out = agents[1 - human_player].step(
            time_step, is_evaluation=True)
        action = agent_out.action
    time_step = env.step([action])

print("End of game!")
if time_step.rewards[human_player] > 0:
    print("You win")
elif time_step.rewards[human_player] < 0:
    print("You lose")
else:
    print("Draw")


You are playing as X (you are second)

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '0' '.']]


Choose an action (empty to print legal actions):  X


Could not parse the action: X


Choose an action (empty to print legal actions):  3X


Could not parse the action: 3X


...


[['X' '.' '.']
 ['X' '.' '0']
 ['0' '0' 'X']]

[['X' '.' '0']
 ['X' '.' '0']
 ['0' '0' 'X']]


Choose an action (empty to print legal actions):  


Legal actions(s):
  1: o(0,1)    4: o(1,1)  


Choose an action (empty to print legal actions):  4


End of game!
You win


# Train agents for the 10000 next episodes
This will make agent much stronger.

In [61]:
train_episodes = 10000

# Train agent
for ep in range(train_episodes):
    time_step = env.reset()
    while not time_step.last():
        player_id = time_step.observations["current_player"]
        agent_output = agents[player_id].step(time_step)
        action_list = [agent_output.action]
        time_step = env.step(action_list)

    # Episode is over, step all agents with final info state.
    for agent in agents:
        agent.step(time_step)
print("Training is over.")

Training is over.


# Evaluate first agent playing agains random agent for 1000 episodes
Reward -1 means loss  
Reward 0 means draw  
Reward 1 mean win

In [26]:
# Evaluate against random agent
random_agents = [
    random_agent.RandomAgent(player_id=idx, num_actions=num_actions)
    for idx in range(num_players)
]
r_mean = eval_against_random_bots(env, agents, random_agents, 1000)
print("Mean episode rewards: %s" % r_mean[0])

Mean episode rewards: 0.941


# Now you can play against trained model
There is a high probability that you will draw with the current agent.

In [62]:
human_player = 1
human_agent = human.HumanBot()

print("You are playing as X (you are second)" ) 
time_step = env.reset()
while not time_step.last():
    player_id = time_step.observations["current_player"]
    print("\n%s" % pretty_board(time_step))

    if player_id == human_player:
        agent_out = agents[human_player].step(time_step, is_evaluation=True)
        action = human_agent.step(env._state)
    else:
        agent_out = agents[1 - human_player].step(
            time_step, is_evaluation=True)
        action = agent_out.action
    time_step = env.step([action])

print("End of game!")
if time_step.rewards[human_player] > 0:
    print("You win")
elif time_step.rewards[human_player] < 0:
    print("You lose")
else:
    print("Draw")


You are playing as X (you are second)

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]

[['.' '0' '.']
 ['.' '.' '.']
 ['.' '.' '.']]


Choose an action (empty to print legal actions):  


Legal actions(s):
  0: o(0,0)    3: o(1,0)    5: o(1,2)    7: o(2,1)  
  2: o(0,2)    4: o(1,1)    6: o(2,0)    8: o(2,2)  


Choose an action (empty to print legal actions):  4



[['.' '0' '.']
 ['.' 'X' '.']
 ['.' '.' '.']]

[['.' '0' '.']
 ['0' 'X' '.']
 ['.' '.' '.']]


...


[['X' '0' '.']
 ['0' 'X' '.']
 ['X' '.' '0']]

[['X' '0' '0']
 ['0' 'X' '.']
 ['X' '.' '0']]


Choose an action (empty to print legal actions):  


Legal actions(s):
  5: o(1,2)    7: o(2,1)  


Choose an action (empty to print legal actions):  5



[['X' '0' '0']
 ['0' 'X' 'X']
 ['X' '.' '0']]
End of game!
Draw


# Let visualize Q-value for some states
Here you can see how certain actions are evaluated by the agent.  
It should take action with the highest value.

You can run this cell multiple times to see different configurations.

In [63]:
def pretty_decoded_board(state_decoded, q_values):
    """Returns the board in `time_step` in a human readable format."""
    info_state = state_decoded
    x_locations = np.nonzero(info_state[9:18])[0]
    o_locations = np.nonzero(info_state[18:])[0]
    board = np.full(3 * 3, "   .   ")
    board[x_locations] = "   X   "
    board[o_locations] = "   O   "
    for index, q_val in q_values.items():
        board[index] = f"{q_val:7.5}"

    board = np.reshape(board, (3, 3))
    return board
    
agent = agents[1 - human_player]

q_values_are_zeros = True
while q_values_are_zeros:
    state_hash = np.random.choice(list(agent._q_values))
    state_decoded = [float(i) for i in state_hash[1: -1].split(",")]  # 27 variables (9 * len([O, X, empty]))
    q_values = agent._q_values[state_hash]
    q_values_are_zeros = (np.array(list(q_values.values())) == 0).all()

print("Sampled state:")
print(pretty_decoded_board(state_decoded, {}))

print("\nQ-Values:")
print(pretty_decoded_board(state_decoded, q_values))


Sampled state:
[['   O   ' '   .   ' '   .   ']
 ['   X   ' '   X   ' '   O   ']
 ['   .   ' '   X   ' '   O   ']]

Q-Values:
[['   O   ' '   0.01' '    0.0']
 ['   X   ' '   X   ' '   O   ']
 ['    0.0' '   X   ' '   O   ']]


##### 